In [50]:
import pandas as pd
import numpy as np
import json
import math
import urllib.request
import dateutil.parser
import dateutil.rrule
import dateutil.tz
import datetime
import re
import gc
import time

In [18]:
tzUTC = dateutil.tz.gettz('UTC')
tzLocal = dateutil.tz.gettz('Europe/London')

earliestData = datetime.datetime.strptime('2020-04-20T00:00:00Z', '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=tzUTC)

In [16]:
visionApiBase = 'https://uo-vision.dev.urbanobservatory.ac.uk/stills/dict'
visionResponse = json.loads(
    urllib.request.urlopen(visionApiBase).read().decode('utf-8')
)

In [30]:
pdSources = pd.DataFrame.from_records(visionResponse).transpose()
pdSources['min_date'] = pdSources['min_date'].apply(lambda d: datetime.datetime.strptime(d, '%Y-%m-%d').replace(tzinfo=tzUTC))
pdSources['max_date'] = pdSources['max_date'].apply(lambda d: datetime.datetime.strptime(d, '%Y-%m-%d').replace(tzinfo=tzUTC))
pdSources

,min_date,max_date
CM_A69A1,2017-10-31 00:00:00+00:00,2020-04-26 00:00:00+00:00
GH_A1114A1,2017-10-31 00:00:00+00:00,2020-04-26 00:00:00+00:00
GH_A167F1,2017-10-31 00:00:00+00:00,2020-04-26 00:00:00+00:00
GH_A167G1,2017-10-31 00:00:00+00:00,2020-04-26 00:00:00+00:00
GH_A167H1,2017-10-31 00:00:00+00:00,2020-04-26 00:00:00+00:00
...,...,...
VAISALACCTV54,2020-04-20 00:00:00+00:00,2020-04-26 00:00:00+00:00
VAISALACCTV56,2020-04-20 00:00:00+00:00,2020-04-26 00:00:00+00:00
VAISALACCTV57,2020-04-20 00:00:00+00:00,2020-04-26 00:00:00+00:00
VAISALACCTV58,2020-04-20 00:00:00+00:00,2020-04-26 00:00:00+00:00


In [51]:
dfPointTs = None

for cameraId in pdSources.index:
    source = pdSources[pdSources.index == cameraId].to_dict(orient='records')[0]
    
    print(cameraId)
    print('  [', end='')
    
    sourceTs = None
    
    for date in dateutil.rrule.rrule(
            dateutil.rrule.DAILY,
            interval=1,
            dtstart=source['min_date'] if source['min_date'] > earliestData else earliestData,
            until=source['max_date']
        ):
        
        windowResponse = None
        windowAttempts = 0
        while windowResponse is None:
            try:
                windowAttempts = windowAttempts + 1
                windowResponse = json.loads(
                    urllib.request.urlopen(
                      'https://uo-vision.dev.urbanobservatory.ac.uk/stills/counts?location=%s&date=%s' % (cameraId, date.isoformat()[0:10])
                    ).read().decode('utf-8')
                )
            except:
                windowAttempts = windowAttempts + 1
                print('x', end='')
                time.sleep(min(windowAttempts, 10))

        sourceOnDay = pd.DataFrame.from_records(pd.json_normalize(windowResponse), index=['ts'])
        sourceOnDay.drop(columns=['camera', 'url'], inplace=True)
        
        if sourceTs is None:
            sourceTs = sourceOnDay
        else:
            sourceTs = sourceTs.append(sourceOnDay)
        
        print('.', end='')
    
    print(']')
    
    sourceTs.rename(inplace=True, errors='ignore', columns={
        'counts.bus': 'Bus',
        'counts.car': 'Car',
        'counts.cyclist': 'Cyclist',
        'counts.motorcyclist': 'Motorcyclist',
        'counts.person': 'Person',
        'counts.truck': 'Truck',
        'counts.van': 'Van'
    })
    sourceTs = sourceTs.add_prefix('%s: ' % cameraId)
    
    if dfPointTs is None:
        dfPointTs = sourceTs
    else:
        dfPointTs = dfPointTs.join(sourceTs)
    
dfPointTs

CM_A69A1
  [x

AttributeError: module 'math' has no attribute 'min'

In [ ]:
dfPointTs

In [ ]:
dfPointTs.to_pickle('../cache/recent-feature-counts-pd.pkl')